# Solving SAT problem with Grover's algorithm

The **"Solving SAT problem with Grover's algorithm"** quantum kata is a series of exercises designed
to get you comfortable with using Grover's algorithm to solve realistic problems
using boolean satisfiability problems (SAT) as an example.

Each task is wrapped in one operation preceded by the description of the task.
Your goal is to fill in the blank (marked with the `// ...` comments)
with some Q# code that solves the task. To verify your answer, run the cell using Ctrl+Enter (⌘+Enter on macOS).

Within each section, tasks are given in approximate order of increasing difficulty; 
harder ones are marked with asterisks.

## Part I. Oracles for SAT problems

The most interesting part of learning Grover's algorithm is solving realistic problems.
This means using oracles which express an actual problem and not simply hard-code a known solution.
In this section we'll learn how to express boolean satisfiability problems as quantum oracles.

### Task 1.1. The AND oracle: $f(x) = x_0 \wedge x_1$

**Inputs:** 

  1. 2 qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

**Goal:**

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2), 
i.e., flip the target state if all qubits of the query register are in the $|1\rangle$ state, 
and leave it unchanged otherwise.

Leave the query register in the same state it started in.

**Stretch Goal:** 

Can you implement the oracle so that it would work 
for `queryRegister` containing an arbitrary number of qubits?

In [1]:
%kata T11_Oracle_And 

operation Oracle_And (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    Controlled X(queryRegister, target);
}

Success!

*Can't come up with a solution? See the explained solution in the [Solving SAT problem with Grover's algorithm Workbook](./Workbook_SolveSATWithGrover.ipynb#Task-1.1.-The-AND-oracle:-$f(x)-=-x_0-\wedge-x_1$).*

### Task 1.2. The OR oracle: $f(x) = x_0 \vee x_1$

**Inputs:** 

  1. 2 qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

**Goal:** 

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2), 
i.e., flip the target state if at least one qubit of the query register is in the $|1\rangle$ state, 
and leave it unchanged otherwise.

Leave the query register in the same state it started in.

**Stretch Goal:** 

Can you implement the oracle so that it would work 
for `queryRegister` containing an arbitrary number of qubits?

In [2]:
%kata T12_Oracle_Or 

open Microsoft.Quantum.Canon;

operation Oracle_Or (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    X(target);
    ControlledOnInt(0, X)(queryRegister, target);
}

Success!

*Can't come up with a solution? See the explained solution in the [Solving SAT problem with Grover's algorithm Workbook](./Workbook_SolveSATWithGrover.ipynb#Task-1.2.-The-OR-oracle:-$f(x)-=-x_0-\vee-x_1$).*

### Task 1.3. The XOR oracle: $f(x) = x_0 \oplus x_1$

**Inputs:** 

  1. 2 qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

**Goal:** 

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2), 
i.e., flip the target state if the qubits of the query register are in different states, 
and leave it unchanged otherwise.

Leave the query register in the same state it started in.

**Stretch Goal:** 

Can you implement the oracle so that it would work 
for `queryRegister` containing an arbitrary number of qubits?

In [3]:
%kata T13_Oracle_Xor 

operation Oracle_Xor (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    for q in queryRegister {
        CNOT(q, target);
    }
}

Success!

*Can't come up with a solution? See the explained solution in the [Solving SAT problem with Grover's algorithm Workbook](./Workbook_SolveSATWithGrover.ipynb#Task-1.3.-The-XOR-oracle:-$f(x)-=-x_0-\oplus-x_1$).*

### Task 1.4. Alternating bits oracle: $f(x) = (x_0 \oplus x_1) \wedge (x_1 \oplus x_2) \wedge \dots \wedge (x_{N-2} \oplus x_{N-1})$

**Inputs:** 

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

**Goal:**

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2).

Leave the query register in the same state it started in.

> This oracle marks two states similar to the state explored in task 1.2 of the GroversAlgorithm kata: 
$|10101...\rangle$ and $|01010...\rangle$.  
It is possible (and quite straightforward) to implement this oracle based on this observation; 
however, for the purposes of learning to write oracles to solve SAT problems we recommend using the representation above.

<details>
  <summary><b>Need a hint? Click here</b></summary>
  Remember that you can use operations defined in previous tasks.
</details>

In [8]:
%kata T14_Oracle_AlternatingBits 

operation Oracle_AlternatingBits (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    let N = Length(queryRegister);
    use anc = Qubit[N-1];
    within {
        for i in 0..N-2 {
            CNOT(queryRegister[i], anc[i]);
            CNOT(queryRegister[i+1], anc[i]);
        }
    }
    apply {
        Controlled X(anc, target);
    }
}

Success!

*Can't come up with a solution? See the explained solution in the [Solving SAT problem with Grover's algorithm Workbook](./Workbook_SolveSATWithGrover.ipynb#Task-1.4.-Alternating-bits-oracle:-$f(x)-=-(x_0-\oplus-x_1)-\wedge-(x_1-\oplus-x_2)-\wedge-\dots-\wedge-(x_{N-2}-\oplus-x_{N-1})$).*

### Task 1.5. Evaluate one clause of a SAT formula

> For SAT problems, $f(x)$ is represented as a conjunction (an AND operation) of several clauses on $N$ variables,
and each clause is a disjunction (an OR operation) of **one or several** variables or negated variables:
>
> $$clause(x) = \bigvee_k y_{k},\text{ where }y_{k} =\text{ either }x_j\text{ or }\neg x_j\text{ for some }j \in \{0, \dots, N-1\}$$
>
> For example, one of the possible clauses on two variables is:
>
> $$clause(x) = x_0 \vee \neg x_1$$

**Inputs:**

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

  3. A 1-dimensional array of tuples `clause` which describes one clause of a SAT problem instance $clause(x)$.

`clause` is an array of one or more tuples, each of them describing one component of the clause.

Each tuple is an `(Int, Bool)` pair:

* the first element is the index $j$ of the variable $x_j$, 
* the second element is `true` if the variable is included as itself ($x_j$) and `false` if it is included as a negation ($\neg x_j$).

**Example:**

* The clause $x_0 \vee \neg x_1$ can be represented as `[(0, true), (1, false)]`.

**Goal:**

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2).

Leave the query register in the same state it started in.

In [9]:
function GetClauseQubits (queryRegister : Qubit[], clause : (Int, Bool)[]) : (Qubit[], Bool[]) {
    mutable qubits = [];
    mutable must_be_flipped = [];

    for (qi, flipped) in clause {
        set qubits += [queryRegister[qi]];
        set must_be_flipped += [not flipped];
    }

    return (qubits, must_be_flipped);
}

GetClauseQubits

In [20]:
%kata T15_Oracle_SATClause 

open Microsoft.Quantum.Arrays;

operation Oracle_SATClause (queryRegister : Qubit[], target : Qubit, clause : (Int, Bool)[]) : Unit is Adj {
    let (qubits, flip) = GetClauseQubits(queryRegister, clause);
    within {
        for (q, f) in Zipped(qubits, flip) {
            if f {
                X(q);
            }
        }
    }
    apply {
        Oracle_Or(qubits, target);
    }
}

Testing SAT clause instance x1...
Testing SAT clause instance x3 ∨ x2...
Testing SAT clause instance x3 ∨ ¬x4 ∨ x2...
Testing SAT clause instance ¬x2 ∨ x0 ∨ x3 ∨ x1...
Testing SAT clause instance x0 ∨ x1 ∨ ¬x2 ∨ x3 ∨ x4...
Testing SAT clause instance ¬x2 ∨ x6 ∨ ¬x5 ∨ ¬x1 ∨ ¬x3 ∨ ¬x0...
Testing SAT clause instance x1 ∨ ¬x0 ∨ ¬x2 ∨ ¬x4 ∨ x3...
Testing SAT clause instance x0 ∨ ¬x1 ∨ ¬x2...
Testing SAT clause instance x2 ∨ x0 ∨ x1...
Testing SAT clause instance ¬x4 ∨ ¬x1 ∨ x3 ∨ ¬x6 ∨ ¬x5 ∨ ¬x0 ∨ ¬x2...


Success!

*Can't come up with a solution? See the explained solution in the [Solving SAT problem with Grover's algorithm Workbook](./Workbook_SolveSATWithGrover.ipynb#Task-1.5.-Evaluate-one-clause-of-a-SAT-formula).*

### Task 1.6. k-SAT problem oracle

> For k-SAT problems, $f(x)$ is represented as a conjunction (an AND operation) of $M$ clauses on $N$ variables,
and each clause is a disjunction (an OR operation) of **one or several** variables or negated variables:
>
> $$f(x) = \bigwedge_i \big(\bigvee_k y_{ik} \big),\text{ where }y_{ik} =\text{ either }x_j\text{ or }\neg x_j\text{ for some }j \in \{0, \dots, N-1\}$$

**Inputs:**

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

  3. A 2-dimensional array of tuples `problem` which describes the k-SAT problem instance $f(x)$.

$i$-th element of `problem` describes the $i$-th clause of $f(x)$; 
it is an array of one or more tuples, each of them describing one component of the clause.

Each tuple is an `(Int, Bool)` pair:

* the first element is the index $j$ of the variable $x_j$, 
* the second element is `true` if the variable is included as itself ($x_j$) and `false` if it is included as a negation ($\neg x_j$).

**Example:**

A more general case of the OR oracle for 3 variables $f(x) = (x_0 \vee x_1 \vee x_2)$ can be represented as `[[(0, true), (1, true), (2, true)]]`.

**Goal:**

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2).

Leave the query register in the same state it started in.

In [22]:
%kata T16_Oracle_SAT 

operation Oracle_SAT (queryRegister : Qubit[], target : Qubit, problem : (Int, Bool)[][]) : Unit is Adj {
    use anc = Qubit[Length(problem)];
    within {
        for (a, clause) in Zipped(anc, problem) {
            Oracle_SATClause(queryRegister, a, clause);
        } 
    }
    apply {
        Oracle_And(anc, target);
    }
}

Testing 2-SAT instance (2, (x0 ∨ x1))...
Testing 2-SAT instance (2, (x0 ∨ x1) ∧ (¬x1 ∨ ¬x0))...
Testing 2-SAT instance (3, (¬x1 ∨ ¬x2) ∧ (x0 ∨ x1) ∧ (¬x1 ∨ ¬x0) ∧ (x2 ∨ x1))...
Testing k-SAT instance (3, (¬x0 ∨ ¬x2 ∨ x1) ∧ (x2))...
Testing k-SAT instance (4, (x2 ∨ x1 ∨ x3 ∨ ¬x0) ∧ (¬x0) ∧ (x2 ∨ ¬x1 ∨ ¬x0))...
Testing k-SAT instance (5, (¬x3 ∨ ¬x1 ∨ x0) ∧ (x4 ∨ x1) ∧ (x2 ∨ x3 ∨ ¬x1) ∧ (¬x2 ∨ ¬x0 ∨ ¬x4 ∨ ¬x3))...
Testing k-SAT instance (6, (x4 ∨ x5) ∧ (x5 ∨ x0 ∨ ¬x3 ∨ ¬x1) ∧ (¬x1 ∨ x3 ∨ ¬x2 ∨ x4) ∧ (x1) ∧ (x2 ∨ x1))...


Success!

*Can't come up with a solution? See the explained solution in the [Solving SAT problem with Grover's algorithm Workbook](./Workbook_SolveSATWithGrover.ipynb#Task-1.6.-k-SAT-problem-oracle).*

## Part II. Oracles for exactly-1 3-SAT problem

Exactly-1 3-SAT problem (also known as "one-in-three 3-SAT") is a variant of a general 3-SAT problem.
It has a structure similar to a 3-SAT problem, but each clause must have *exactly one* true literal, 
while in a normal 3-SAT problem each clause must have *at least one* true literal.

### Task 2.1. "Exactly one $|1\rangle$" oracle

**Inputs:** 

  1. 3 qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

**Goal:** 

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2), 
where $f(x) = 1$ if exactly one bit of $x$ is in the $|1\rangle$ state, and $0$ otherwise.

Leave the query register in the same state it started in.

**Stretch Goal:** 

Can you implement the oracle so that it would work 
for `queryRegister` containing an arbitrary number of qubits?

In [52]:
%kata T21_Oracle_Exactly1One 

operation Oracle_Exactly1One (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    for i in 0..Length(queryRegister) - 1 {
        ControlledOnInt(2^i, X)(queryRegister, target);
    }
}

Success!

*Can't come up with a solution? See the explained solution in the [Solve SAT problem with Grover's algorithm Workbook](./Workbook_SolveSATWithGrover.ipynb#Task-2.1.-"Exactly-one-$|1\rangle$"-oracle).*

### Task 2.2. "Exactly-1 3-SAT" oracle

**Inputs:**

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

  3. A 2-dimensional array of tuples `problem` which describes the 3-SAT problem instance $f(x)$.

`problem` describes the problem instance in the same format as in task 1.6;
each clause of the formula is guaranteed to have exactly 3 terms.

**Goal:**

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2).

Leave the query register in the same state it started in.

**Example:**

An instance of the problem $f(x) = (x_0 \vee x_1 \vee x_2)$ can be represented as `[[(0, true), (1, true), (2, true)]]`,
and its solutions will be `(true, false, false)`, `(false, true, false)` and `(false, false, true)`, 
but none of the variable assignments in which more than one variable is true, which are solutions for the general SAT problem.

<br/>
<details>
  <summary><b>Need a hint? Click here</b></summary>
  Can you reuse parts of the code in section 1?
</details>

In [53]:
function GetClauseQubits2(queryRegister: Qubit[], problem: (Int, Bool)[]): Qubit[] {
    mutable qs = [];

    for (i, _) in problem {
        set qs += [queryRegister[i]];
    }

    return qs;
}

GetClauseQubits2

In [54]:
open Microsoft.Quantum.Diagnostics;

operation Oracle_Exactly1_3SAT_Clause(queryRegister: Qubit[], target: Qubit, problem: (Int, Bool)[]): Unit is Adj {
    within {
        for (idx, pos) in problem {
            if not pos {
                X(queryRegister[idx]);
            }
        }
    }
    apply {
        let qs = GetClauseQubits2(queryRegister, problem);
        Oracle_Exactly1One(qs, target);
    }
}

Oracle_Exactly1_3SAT_Clause

In [55]:
%kata T22_Oracle_Exactly1SAT 

operation Oracle_Exactly1_3SAT (queryRegister : Qubit[], target : Qubit, problem : (Int, Bool)[][]) : Unit is Adj {
    let n_clauses = Length(problem);
    use anc = Qubit[n_clauses];
    within {
        for i in 0..n_clauses-1 {
            Oracle_Exactly1_3SAT_Clause(queryRegister, anc[i], problem[i]);
        }
    }
    apply {
        Oracle_And(anc, target);
    }
}

Testing exactly-1 3-SAT instance (2, (¬x0 ∨ x1))...
Testing exactly-1 3-SAT instance (3, (¬x1 ∨ x2 ∨ x0) ∧ (x1 ∨ ¬x2 ∨ x0))...
Testing exactly-1 3-SAT instance (4, (¬x1 ∨ ¬x3 ∨ ¬x0) ∧ (¬x1 ∨ ¬x3 ∨ ¬x0) ∧ (x0 ∨ x3 ∨ x1))...
Testing exactly-1 3-SAT instance (5, (x3 ∨ x4 ∨ x2) ∧ (¬x3 ∨ ¬x2 ∨ ¬x1) ∧ (x1 ∨ ¬x3 ∨ x0) ∧ (¬x1 ∨ x3 ∨ ¬x4))...
Testing exactly-1 3-SAT instance (6, (x2 ∨ ¬x5 ∨ ¬x1) ∧ (¬x2 ∨ x1 ∨ ¬x4) ∧ (¬x3 ∨ x1 ∨ x0) ∧ (¬x5 ∨ ¬x2 ∨ x4) ∧ (¬x1 ∨ ¬x2 ∨ x3))...


Success!

*Can't come up with a solution? See the explained solution in the [Solve SAT problem with Grover's algorithm Workbook](./Workbook_SolveSATWithGrover.ipynb#Task-2.2.-"Exactly-1-3-SAT"-oracle).*

## Part III. Using Grover's algorithm for problems with multiple solutions

### Task 3.1. Using Grover's algorithm

**Goal:**

Implement Grover's algorithm and use it to find solutions to SAT instances from parts I and II. 

> If you want to learn the Grover's algorithm itself, try doing [GroversAlgorithm kata](./../GroversAlgorithm/GroversAlgorithm.ipynb) first.

> This is an open-ended task, and is not covered by a unit test. To run the code, execute the cell with the definition of the `Run_GroversSearch_Algorithm` operation first; if it compiled successfully without any errors, you can run the operation by executing the next cell (`%simulate Run_GroversSearch_Algorithm`).

> Note that this task relies on your implementations of the previous tasks. If you are getting the "No variable with that name exists." error, you might have to execute previous code cells before retrying this task.

<details>
  <summary><b>Need a hint? Click here</b></summary>
Experiment with SAT instances with different number of solutions and the number of algorithm iterations 
to see how the probability of the algorithm finding the correct answer changes depending on these two factors.

For example, 
* the AND oracle from task 1.1 has exactly one solution,
* the alternating bits oracle from task 1.4 has exactly two solutions,
* the OR oracle from task 1.2 for 2 qubits has exactly 3 solutions, and so on.
</details>

In [66]:
open Microsoft.Quantum.Measurement;

operation OracleConverterImpl_Reference (markingOracle : ((Qubit[], Qubit) => Unit is Adj), register : Qubit[]) : Unit is Adj {

    use target = Qubit();
    within {
        // Put the target into the |-⟩ state, perform the apply functionality, then put back into |0⟩ so we can return it
        X(target);
        H(target);
    }
    apply {
        // Apply the marking oracle; since the target is in the |-⟩ state,
        // flipping the target if the register satisfies the oracle condition will apply a -1 factor to the state
        markingOracle(register, target);
    }
}

function OracleConverter_Reference (markingOracle : ((Qubit[], Qubit) => Unit is Adj)) : (Qubit[] => Unit is Adj) {
    return OracleConverterImpl_Reference(markingOracle, _);
}

operation GroversAlgorithm_Loop (register : Qubit[], oracle : ((Qubit[], Qubit) => Unit is Adj), iterations : Int) : Unit {
    let phaseOracle = OracleConverter_Reference(oracle);
    ApplyToEach(H, register);
    for i in 1 .. iterations {
        phaseOracle(register);
        within {
            ApplyToEachA(H, register);
            ApplyToEachA(X, register);
        }
        apply {
            Controlled Z(Most(register), Tail(register));
        }
    }
}

operation Run_GroversSearch_Algorithm () : Unit {
    let problem = [[(1, false), (2, false)], [(0, true), (1, true)], [(1, false), (0, false)], [(2, true), (1, true)]];
    let oracle = Oracle_SAT(_, _, problem);
    use qubits = Qubit[3];
    GroversAlgorithm_Loop(qubits, oracle, 3);
    let res = MultiM(qubits);
    Message($"{res}");
}

OracleConverterImpl_Reference, OracleConverter_Reference, GroversAlgorithm_Loop, Run_GroversSearch_Algorithm

In [70]:
%simulate Run_GroversSearch_Algorithm

[One,One,One]


()

*Can't come up with a solution? See the explained solution in the [Solve SAT problem with Grover's algorithm Workbook](./Workbook_SolveSATWithGrover.ipynb#Task-3.1.-Using-Grover's-algorithm).*

### Task 3.2. Universal implementation of Grover's algorithm

**Inputs:**

  1. The number of qubits N.

  2. A marking oracle which implements a boolean expression, similar to the oracles from part I.

**Output:**

An array of N boolean values which satisfy the expression implemented by the oracle 
(i.e., any basis state marked by the oracle).

> Note that the similar task in the [GroversAlgorithm kata](./../GroversAlgorithm/GroversAlgorithm.ipynb) required you to implement Grover's algorithm 
in a way that would be robust to accidental failures, but you knew the optimal number of iterations 
(the number that minimized the probability of such failure).
> 
> In this task you also need to make your implementation robust to not knowing the optimal number of iterations.
> You can see an example of doing that in [Exploring Grover's Algorithm tutorial](./../tutorials/ExploringGroversAlgorithm/ExploringGroversAlgorithm.ipynb).

In [73]:
%kata T32_UniversalGroversAlgorithm 

open Microsoft.Quantum.Convert;

operation UniversalGroversAlgorithm (N : Int, oracle : ((Qubit[], Qubit) => Unit is Adj)) : Bool[] {
    mutable answer = [false, size = N];
    use (register, test) = (Qubit[N], Qubit());
    mutable correct = false;
    mutable iter = 1;
    repeat {
        GroversAlgorithm_Loop(register, oracle, iter);
        set answer = ResultArrayAsBoolArray(MultiM(register));
        oracle(register, test);
        if MResetZ(test) == One {
            set correct = true;
        }
        ResetAll(register);
    }
    until (correct or iter > 10)
    fixup {
        set iter *= 2;
    }
    if (not correct) {
        fail "Failure";
    }
    return answer;
}

Success!

*Can't come up with a solution? See the explained solution in the [Solve SAT problem with Grover's algorithm Workbook](./Workbook_SolveSATWithGrover.ipynb#Task-3.2.-Universal-implementation-of-Grover's-algorithm).*